 - 修改./conf/hdfsutils.conf
 - 替换./conf/hdfs.keytab

In [1]:
import pandas as pd
from imp import reload
import sys

import os

sys.path.append('../..')
from aibox import hdfsutils
from aibox import dbutils
reload(dbutils)
reload(hdfsutils)


<module 'aibox.hdfsutils' from '../../aibox/hdfsutils.py'>

# HDFS常用命令

## 初始化HDFS对象

In [2]:
hdfs_client = hdfsutils.init_hdfs()

tdh_principal: hdfs/dm46@TDH
hdfs_keytab_path: /home/kwq/jupyter/aibox/./conf/hdfs.keytab
kinti status: kinit success.
active nm: http://172.26.0.47:50070


## ls

In [3]:
hdfsutils.hdfs_ls('/', hdfs_client)

['$Recycle.Bin',
 '2、智能小车底盘安装之电池盒固定.mp4',
 '3、上电测试和电池使用说明.mp4',
 'Arduino小车视频',
 'SamsungPortableSSD_Setup_Win_1.0.exe',
 'Winchk.exe',
 'ba.txt',
 'desktop.ini',
 'discover1',
 'discover2',
 'discover3',
 'es',
 'fide',
 'hdfs-client-core.exe4j',
 'home',
 'hyperbase1',
 'hyperbase2',
 'inceptor1',
 'inceptor2',
 'inceptor3',
 'inceptor4',
 'ljr',
 'nameservice1',
 'pack',
 'rhel-server-6.3-i386-dvd.iso',
 'slipstream1',
 'slipstream2',
 'slipstreamstudio1',
 'sohpon',
 'sophon',
 'testhdfs',
 'tmp',
 'unins000.dat',
 'unins000.exe',
 'uninstall.bat',
 'user',
 'winpeshl.ini',
 'yarn1',
 'zstp']

## mkdir


In [4]:
hdfsutils.hdfs_mkdir('/tmp/hdfstest', hdfs_client)

## put

In [5]:
hdfs_file_name = '/tmp/hdfstest/hdfsutils.ipynb'
local_file_name = './hdfsutils.ipynb'
    
hdfsutils.hdfs_put(hdfs_file_name, local_file_name, hdfs_client)

In [6]:
hdfsutils.hdfs_ls('/tmp/hdfstest', hdfs_client)

['hdfsutils.ipynb']

## download

In [7]:
source_file = '/tmp/hdfstest/hdfsutils.ipynb'
target_path = '/tmp'

if os.path.exists(target_path + '/hdfsutils.ipynb'):
    os.remove(target_path + '/hdfsutils.ipynb')
    
hdfsutils.hdfs_download(source_file, target_path, hdfs_client)

In [8]:
# 检查
!ls -l /tmp/ | grep hdfsutils

-rw-r--r--.  1 root     root         9517 Feb 19 02:05 hdfsutils.ipynb


## delete

In [9]:
hdfs_file_name =  '/tmp/hdfstest/hdfsutils.ipynb'
hdfsutils.hdfs_delete(hdfs_file_name, hdfs_client)
    

In [10]:
hdfsutils.hdfs_ls('/tmp/hdfstest', hdfs_client)

[]

# HDFS+CSV表

## 建外表

drop table default.t_city_info_csv;
create table default.t_city_info_csv(
    idx string comment '唯一列',
    city_name string comment '城市名称',
    gdp double comment 'gdp增速'
)
stored as csvfile
location '/tmp/t_city_info_csv';

## 准备数据

In [11]:
data = pd.DataFrame({'idx':[1,2,3,4,5], 
                     'city_name':['beijing', 'shanghai', 'guangzhou', 'shenzhen', 'chongqing'],
                     'gdp':[8.5, 8.1, 8.1, 6.7, 8.3]
                    })

# 构造数据
data.to_csv('./data/t_city_info.csv', index=False)

data

,idx,city_name,gdp
0,1,beijing,8.5
1,2,shanghai,8.1
2,3,guangzhou,8.1
3,4,shenzhen,6.7
4,5,chongqing,8.3


## 上传文件

In [12]:
hdfs_file_name = '/tmp/t_city_info_csv/t_city_info_csv.csv'
local_file_name = './data/t_city_info.csv'

# 1. 初始化hdfs
hdfs_client = hdfsutils.init_hdfs()


# 2. 检查文件是否存在，如果存在，先删除
file_check = hdfsutils.hdfs_ls('/tmp/t_city_info_csv/', hdfs_client)
if file_check != []:
    print('File exists, delete first:', hdfs_file_name)
    hdfsutils.hdfs_delete(hdfs_file_name, hdfs_client)
    
# 3. 上传文件
hdfsutils.hdfs_put(hdfs_file_name, local_file_name, hdfs_client)

tdh_principal: hdfs/dm46@TDH
hdfs_keytab_path: /home/kwq/jupyter/aibox/./conf/hdfs.keytab
kinti status: kinit success.
active nm: http://172.26.0.47:50070
File exists, delete first: /tmp/t_city_info_csv/t_city_info_csv.csv


## 查询验证

In [13]:
dbutils.selectHiveByPyhive("select * from default.t_city_info_csv order by idx asc")

,idx,city_name,gdp
0,1,beijing,8.5
1,2,shanghai,8.1
2,3,guangzhou,8.1
3,4,shenzhen,6.7
4,5,chongqing,8.3
5,idx,city_name,NaN
